<a href="https://colab.research.google.com/github/aishwarya-kumar/skillrec_for_gigworkers/blob/main/RAG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install langchain-community chromadb pypdf pdfplumber
# !pip install pypdf

In [64]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.errors import NotFoundError
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdfplumber
import os
import re
from transformers import pipeline

In [5]:
# Load the SentenceTransformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize ChromaDB client
client = chromadb.Client()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# # Function to load documents from a directory (PDF files)
# def load_documents(path):
#     document_loader = PyPDFDirectoryLoader(path)
#     return document_loader.load()

In [29]:
def load_documents(path):
    documents = []
    print(f"Starting to load PDFs from directory: {path}")

    # Loop through all files in the directory and process PDFs
    for file in os.listdir(path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(path, file)
            print(f"Processing file: {pdf_path}")

            with pdfplumber.open(pdf_path) as pdf:
                text = ""

                # Extract text from each page of the PDF
                for i, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                    else:
                        print(f"Warning: No text found on page {i+1} of {file}")

                # If there's any text extracted, add it to the documents list
                if text.strip():
                    documents.append({"page_content": text, "metadata": {"source": file}})
                    print(f"Extracted text from {file}")
                else:
                    print(f"Warning: No text extracted from {file}. Skipping.")

    # Check if any documents were loaded
    if not documents:
        raise ValueError("No content extracted from any PDF in the directory.")

    print(f"Loaded {len(documents)} documents successfully from {path}.")
    return documents

In [18]:
# # Function to preprocess the text into chunks
# def preprocess_text(docs, chunk_size, chunk_overlap):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         length_function=len,  # Function to compute the length of the text
#         add_start_index=True,  # Flag to add start index to each chunk
#     )
#     chunks = text_splitter.split_documents(docs)
#     print(f"Split {len(docs)} documents into {len(chunks)} chunks.")
#     return chunks

In [27]:
# def preprocess_text(docs, chunk_size, chunk_overlap):
#     if not docs:
#         raise ValueError("No documents found to preprocess.")

#     print(f"Starting text preprocessing. Chunk size: {chunk_size}, Chunk overlap: {chunk_overlap}")

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap,
#         length_function=len,  # Function to compute the length of the text
#         add_start_index=True,  # Flag to add start index to each chunk
#     )

#     # Split documents into chunks
#     chunks = text_splitter.split_documents(docs)
#     print(f"Split {len(docs)} documents into {len(chunks)} chunks.")

#     return chunks

In [38]:
def preprocess_text(docs, chunk_size, chunk_overlap):
    # Function to clean the extracted text
    def clean_text(text):
        # Remove unwanted newlines, excessive spaces, and carriage returns
        text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
        text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single newline
        text = text.strip()  # Remove leading/trailing whitespace
        return text

    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # Maximum size of each chunk
        chunk_overlap=chunk_overlap,  # How much overlap between chunks
        length_function=len,  # Function to compute the length of the text
        add_start_index=True,  # Add a starting index for each chunk
    )

    all_chunks = []

    for doc in docs:
        page_content = doc['page_content']

        # Clean the text before splitting into chunks
        cleaned_content = clean_text(page_content)

        # Split the cleaned content into chunks
        chunks = text_splitter.split_text(cleaned_content)

        # Add the chunks to the list with metadata
        for chunk in chunks:
            all_chunks.append({
                "page_content": chunk,
                "metadata": doc['metadata']
            })

    print(f"Split {len(docs)} documents into {len(all_chunks)} chunks.")
    return all_chunks

In [8]:
# # Step 3: Generate embeddings using SentenceTransformer
# def get_embeddings(documents):
#     embeddings = model.encode([doc.page_content for doc in documents])  # Access the text through `.page_content`
#     return embeddings


In [46]:
def get_embeddings(chunks):
    print("Starting to generate embeddings for the chunks...")

    # Check if there are any chunks
    if not chunks:
        print("No chunks found! Exiting.")
        return []

    # Extracting page content from chunks and generating embeddings
    page_contents = [chunk['page_content'] for chunk in chunks]
    print(f"Extracted {len(page_contents)} page contents from the chunks.")

    # Generate embeddings using the model
    try:
        print("Generating embeddings using the SentenceTransformer model...")
        embeddings = model.encode(page_contents)
        print(f"Generated embeddings for {len(page_contents)} chunks.")
    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return []

    return embeddings

In [9]:
# def build_chromadb_index(documents, embeddings):
#     collection_name = "tech_jobs"

#     # Try to delete the collection if it exists
#     try:
#         client.delete_collection(name=collection_name)
#         print(f"Deleted existing collection '{collection_name}'.")
#     except chromadb.errors.CollectionNotFoundError:
#         print(f"No existing collection named '{collection_name}' found.")

#     # Create a new collection
#     collection = client.create_collection(name=collection_name)

#     # Ensure there is content before adding to ChromaDB
#     documents_text = [doc.page_content for doc in documents]
#     if not documents_text:
#         raise ValueError("No valid text content found in documents.")

#     # Add the documents and their embeddings to the collection
#     collection.add(
#         documents=documents_text,
#         embeddings=embeddings,
#         metadatas=[{"source": "pdf"}] * len(documents),
#         ids=[str(i) for i in range(len(documents))]
#     )
#     print(f"Added {len(documents)} documents to ChromaDB collection.")

#     return collection

In [60]:
def build_chromadb_index(documents, embeddings):
    collection_name = "tech_jobs"

    # Check if the collection exists
    collections = client.list_collections()  # Get a list of all collections
    if collection_name in collections:
        # If the collection exists, delete it
        client.delete_collection(name=collection_name)
        print(f"Deleted existing collection '{collection_name}'.")
    else:
        print(f"No existing collection named '{collection_name}', proceeding to create a new one.")

    # Create a new collection
    collection = client.create_collection(name=collection_name)
    print(f"Created a new collection '{collection_name}'.")

    # Ensure there is content before adding to ChromaDB
    documents_text = [doc['page_content'] for doc in documents]  # Access 'page_content' correctly
    if not documents_text:
        raise ValueError("No valid text content found in documents.")

    # Add the documents and their embeddings to the collection
    collection.add(
        documents=documents_text,
        embeddings=embeddings,
        metadatas=[{"source": "pdf"}] * len(documents),
        ids=[str(i) for i in range(len(documents))]
    )
    print(f"Added {len(documents)} documents to ChromaDB collection.")

    return collection

In [10]:
# Step 5: Retrieve relevant chunks using ChromaDB
def retrieve_relevant_chunks(query, collection):
    # Generate embedding for the query using the same model
    query_embedding = model.encode([query]).tolist()  # Embedding for the query text

    # Perform similarity search using ChromaDB
    query_result = collection.query(query_embeddings=query_embedding, n_results=3)

    return query_result['documents']


In [68]:
def query_refiner(conversation, query):
    model_name = "google/flan-t5-base"
    refiner = pipeline("text2text-generation", model=model_name)

    prompt = f"Refine the following query based on the conversation context (focusing on career advice and skills for gig workers):\n{conversation}\nQuery: {query}\nRefined Query:"

    refined_query = refiner(prompt)[0]['generated_text']
    return refined_query


In [22]:
# def generate_response(retrieved_chunks, query, conversation_history):

#     # Flatten the retrieved chunks (if it's a list of lists) into a single list of strings
#     if isinstance(retrieved_chunks[0], list):
#         retrieved_chunks = [item for sublist in retrieved_chunks for item in sublist]

#     # Combine retrieved chunks to create context
#     context = " ".join(retrieved_chunks)

#     # Update the conversation history with user query and the context
#     conversation_history.append(f"User: {query}")
#     conversation_history.append(f"Context: {context}")

#     # Refine the query based on the conversation history and user query
#     refined_query = query_refiner("\n".join(conversation_history), query)

#     # Load the response generation model (Flan-T5 or any other available model)
#     model_name = "google/flan-t5-base"
#     generator = pipeline("text2text-generation", model=model_name)

#     # Generate the final response using the refined query and context
#     prompt = f"Context: {context}\nRefined Query: {refined_query}\nAnswer:"
#     response = generator(prompt)[0]['generated_text']

#     # Update conversation history with the bot's response
#     conversation_history.append(f"Bot: {response}")

#     return response, conversation_history

In [12]:
def generate_response(retrieved_chunks, query, conversation_history):

    # Flatten the retrieved chunks (if it's a list of lists) into a single list of strings
    if isinstance(retrieved_chunks[0], list):
        retrieved_chunks = [item for sublist in retrieved_chunks for item in sublist]

    # Combine retrieved chunks to create context
    context = " ".join(retrieved_chunks)

    # Update the conversation history with user query and the context
    conversation_history.append(f"User: {query}")
    conversation_history.append(f"Context: {context}")

    # Refine the query based on the conversation history and user query
    refined_query = query_refiner("\n".join(conversation_history), query)

    # Load the response generation model (Flan-T5 or any other available model)
    model_name = "google/flan-t5-base"
    generator = pipeline("text2text-generation", model=model_name)

    # Structured prompt with required format
    prompt = f"""
    Based on the provided context, structure your response as follows:

    Here are the top 3 job roles/careers for gig workers or freelancers:
    1. [Role]
    2. [Role]
    3. [Role]

    Here are the top 5 skills needed for each:
    1. [Role]: top 5 skills:
       a. [Skill]
       b. [Skill]
       c. [Skill]
       d. [Skill]
       e. [Skill]

    Context: {context}
    Refined Query: {refined_query}
    Answer:
    """
    response = generator(prompt, max_length=500, temperature=0.3)[0]['generated_text']

    # Update conversation history with the bot's response
    conversation_history.append(f"Bot: {response}")

    return response, conversation_history

In [ ]:
# # Get the conversation string to maintain context (for query refinement)
# def get_conversation_string():
#     conversation_string = ""
#     for i in range(len(st.session_state['responses'])-1):
#         conversation_string += "Human: "+st.session_state['requests'][i] + "\n"
#         conversation_string += "Bot: "+ st.session_state['responses'][i+1] + "\n"
#     return conversation_string

In [13]:
def rag_pipeline(pdf_path, query):
    # Initialize conversation history
    conversation_history = []

    # Step 1: Load and extract text from the PDF
    documents = load_documents(pdf_path)

    # Step 2: Preprocess and split text into chunks using LangChain
    chunks = preprocess_text(documents)

    # Step 3: Generate embeddings for the documents using SentenceTransformer
    document_embeddings = get_embeddings(chunks)

    # Step 4: Build ChromaDB index and store documents
    collection = build_chromadb_index(chunks, document_embeddings)

    # Step 5: Retrieve relevant chunks for the user's query
    relevant_chunks = retrieve_relevant_chunks(query, collection)

    # Step 6: Generate a response using Hugging Face's Flan-T5 model
    response, conversation_history = generate_response(relevant_chunks, query, conversation_history)

    return response, conversation_history

In [16]:
from datetime import datetime
# Get the current year and month
current_year = datetime.now().year
current_month = datetime.now().strftime("%B")

pdf_path = "/content/"  # Replace with the path to your PDF directory


In [17]:
# Construct the query dynamically
query_prompt = f"""
Identify the top 3 job roles or careers for gig workers or freelancers in {current_month} {current_year}.
For each role, provide a list of the top 5 most in-demand skills required to succeed.
Structure your response as follows:

Here are the top 3 job roles/careers for gig workers or freelancers in {current_month} {current_year}:
1. [Role 1]
2. [Role 2]
3. [Role 3]

Here are the top 5 skills needed for each role:
1. [Role 1]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]

2. [Role 2]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]

3. [Role 3]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]
"""

conversation_history = []

In [30]:
documents = load_documents(pdf_path)

Starting to load PDFs from directory: /content/
Processing file: /content/SSiTechJobsJune2024One-Pager.pdf
Extracted text from SSiTechJobsJune2024One-Pager.pdf
Processing file: /content/The-Job-Skills-of-2024-Report.pdf
Extracted text from The-Job-Skills-of-2024-Report.pdf
Processing file: /content/WEF_Future_of_Jobs_2023.pdf
Extracted text from WEF_Future_of_Jobs_2023.pdf
Processing file: /content/2024_Work_Trend_Index_Annual_Report_6_7_24_666b2e2fafceb.pdf
Extracted text from 2024_Work_Trend_Index_Annual_Report_6_7_24_666b2e2fafceb.pdf
Processing file: /content/GSR_2024_-_NAMER_English.pdf
Extracted text from GSR_2024_-_NAMER_English.pdf
Processing file: /content/comptia-state-of-the-tech-workforce-2024.pdf
Extracted text from comptia-state-of-the-tech-workforce-2024.pdf
Loaded 6 documents successfully from /content/.


In [35]:
documents[0]

{'page_content': 'Tech Employment\nTrends\nJune 2024 6-month trends\nSnapshot: Tech Employment Top Labor Categories (SSi)\nWhere did SSi place consultants YTD 2024?\n• Current tech unemployment rate dropped to\n2.5% vs. 4% national rate.\nAML Project 19.05%\n• Who’s in demand? AI skills were highest in\ndemand across 12% of postings. BLS notes\nBusiness Professional 13.23%\nthe fastest growing tech occupations over the\nnext decade are: Data Scientists, Infosec IT 10.05%\nAnalysts, Software Developers, Computer\nand Info Research Scientists. Aerospace 9.52%\nTechnical 7.41%\nState of Job Postings\n• Biggest jump in job postings: Data Scientists,\nEngineering 6.88%\nDatabase Admins, Software and Web\nDevelopers. Professional 5.82%\n• As of May, the total number of active job\npostings is 427,000.\nTop Jobs Filled (SSi)\nWhere Are the Tech Jobs?\n• AML Investigation Analyst\n• Integration Engineer\nAccording to CompTIA Monthly Tech Jobs Report\n• Scrum Master\n(May), the top States for T

In [39]:
chunks = preprocess_text(documents, chunk_size= 500, chunk_overlap= 100)

Split 6 documents into 3757 chunks.


In [40]:
chunks[0]

{'page_content': 'Tech Employment Trends June 2024 6-month trends Snapshot: Tech Employment Top Labor Categories (SSi) Where did SSi place consultants YTD 2024? • Current tech unemployment rate dropped to 2.5% vs. 4% national rate. AML Project 19.05% • Who’s in demand? AI skills were highest in demand across 12% of postings. BLS notes Business Professional 13.23% the fastest growing tech occupations over the next decade are: Data Scientists, Infosec IT 10.05% Analysts, Software Developers, Computer and Info',
 'metadata': {'source': 'SSiTechJobsJune2024One-Pager.pdf'}}

In [47]:
document_embeddings = get_embeddings(chunks)

Starting to generate embeddings for the chunks...
Extracted 3757 page contents from the chunks.
Generating embeddings using the SentenceTransformer model...
Generated embeddings for 3757 chunks.


In [61]:
collection = build_chromadb_index(chunks, document_embeddings)

No existing collection named 'tech_jobs', proceeding to create a new one.
Created a new collection 'tech_jobs'.
Added 3757 documents to ChromaDB collection.


In [62]:
relevant_chunks = retrieve_relevant_chunks(query_prompt, collection)

In [67]:
relevant_chunks

[['Skill Trends | NA 2001 02 03 04 Country spotlight Domain rankings Top skills Top target roles United States 70 71 55 1. Geovisualization 1. Personal Financial Advisor Business Tech Data science 2. SQL 2. Business Intelligence Analyst 3. Business Communication 3. Bookkeeper 4. General Accounting 4. Risk Analyst 27.7 M 69 36 Enrollment trends 5. Epidemiology 5. Technology Consultant GenAI Coursera Global Median 6. Bayesian Statistics 6. Network Engineer learners rank age 1,058% YoY enrollments 7.',
  'skills to be core skills for their workers (%) j k l m n o p q r 11 44 10 10 4 26 14 18 12 12 18 23 20 18 8 8 19 18 14 8 10 15 18 14 15 23 12 7 21 9 15 12 17 16 15 11 26 52 48 55 54 44 53 55 53 46 36 50 60 59 50 52 41 36 54 44 62 55 36 49 43 36 44 36 41 46 25 47 39 24 48 48 49 44 71 65 52 52 42 55 56 35 57 53 30 53 44 28 63 66 37 44 43 45 48 54 55 55 50 39 54 54 35 53 54 36 54 57 31 32 38 50 50 44 41 50 53 38 43 45 25 65 46 36 53 43 40 44 33 50 40 49 42 41 44 36 54 37 45 41 35 44 41 30 2

In [65]:
response, conversation_history = generate_response(relevant_chunks, query_prompt, conversation_history)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1654 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [66]:
print("Response:", response)
print("Updated Conversation History:")
for entry in conversation_history:
    print(entry)

Response: 1. [Role] 2. [Role] 3. [Role]
Updated Conversation History:
User: 
Identify the top 3 job roles or careers for gig workers or freelancers in November 2024. 
For each role, provide a list of the top 5 most in-demand skills required to succeed. 
Structure your response as follows:

Here are the top 3 job roles/careers for gig workers or freelancers in November 2024:
1. [Role 1]
2. [Role 2]
3. [Role 3]

Here are the top 5 skills needed for each role:
1. [Role 1]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]

2. [Role 2]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]

3. [Role 3]: top 5 skills:
   a. [Skill 1]
   b. [Skill 2]
   c. [Skill 3]
   d. [Skill 4]
   e. [Skill 5]

Context: Skill Trends | NA 2001 02 03 04 Country spotlight Domain rankings Top skills Top target roles United States 70 71 55 1. Geovisualization 1. Personal Financial Advisor Business Tech Data science 2. SQL 2. B